In [30]:
import numpy as np

def differential_evolution(cost_func, bounds, pop_size=20, mutation=0.8, crossover=0.7, generations=200):
    dimensions = len(bounds)
    pop = np.random.rand(pop_size, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([cost_func(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    
    for i in range(generations):
        for j in range(pop_size):
            idxs = [idx for idx in range(pop_size) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = np.clip(a + mutation * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossover
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = cost_func(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
    return best, fitness[best_idx]


In [34]:
def differential_evolution_for_max(cost_func, bounds, pop_size=20, mutation=0.8, crossover=0.7, generations=200):
    dimensions = len(bounds)
    pop = np.random.rand(pop_size, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([cost_func(ind) for ind in pop_denorm])
    best_idx = np.argmax(fitness)
    best = pop_denorm[best_idx]
    
    for i in range(generations):
        for j in range(pop_size):
            idxs = [idx for idx in range(pop_size) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = np.clip(a + mutation * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossover
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = cost_func(trial_denorm)
            if f > fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f > fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
    return best, fitness[best_idx]

In [31]:
def cost_function(params):
    # Unpack parameters
    Nx, Ny, Lx, Ly, h, Nr, Lr, A_grid, t_rev = params

    # Material costs (per unit)
    c_conductor = 0  # $/m
    c_excavation = 261.59 # $/m³
    c_welding = 12  # $/joint
    c_rod = 0  # $/m
    c_rev = 25  # $/m³

    # Cost components
    C1 = (Nx * Lx + Ny * Ly) * c_conductor
    C2 = (Nx * Lx + Ny * Ly) * h * c_excavation
    C3 = (Nx + Ny) * c_welding
    C4 = Nr * Lr * c_rod
    C5 = A_grid * t_rev * c_rev
    C6 = A_grid * h * c_excavation

    # Total cost
    C_total = C1 + C2 + C3 + C4 + C5 + C6
    return C_total


In [32]:
bounds = [
    (58, 134),  # conductors per direction over a 400 m span
    (58, 134),  # conductors per direction over a 400 m span
    (402.34, 402.34),    # Lx: exact grid side length in X (m)
    (402.34, 402.34),    # Ly: exact grid side length in Y (m)
    (0.5, 1.0),          # h: burial depth (m) per IEEE 80, 0.3–0.5 m typical, 0.5–1 m for 345 kV :contentReference[oaicite:2]{index=2}
    (100, 200),          # Nr: number of rods (≈ perimeter/15 m + equipment ties) :contentReference[oaicite:3]{index=3}
    (2.44, 7.5),         # Lr: rod length (m), 8 ft (2.44 m) standard to 7.5 m for deep soils :contentReference[oaicite:4]{index=4}
    (161874.0, 161874.0),# A_grid: fixed yard area (m²) = 40 ac × 4046.856 m²/ac
    (0.10, 0.20)         # t_rev: revetment thickness (m), ~6 in (0.15 m) per IEEE 80 :contentReference[oaicite:5]{index=5}
]


In [33]:
best_params, min_cost = differential_evolution(cost_function, bounds)
print("Optimal Parameters:", best_params)
print("Minimum Cost: $", min_cost)

Optimal Parameters: [5.80000000e+01 5.80000000e+01 4.02340000e+02 4.02340000e+02
 5.00000000e-01 1.00000000e+02 5.24122546e+00 1.61874000e+05
 1.00000000e-01]
Minimum Cost: $ 27682777.824799996


In [35]:
best_params, max_cost = differential_evolution_for_max(cost_function, bounds)
print("Optimal Parameters:", best_params)
print("Maximum Cost: $", max_cost)

Optimal Parameters: [1.34000000e+02 1.34000000e+02 4.02340000e+02 4.02340000e+02
 1.00000000e+00 1.16709332e+02 7.50000000e+00 1.61874000e+05
 2.00000000e-01]
Maximum Cost: $ 71363701.98079999
